In [1]:
!pip install mtcnn
import os
import cv2
import numpy as np
from tqdm import tqdm
from mtcnn import MTCNN

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 16.8 MB/s eta 0:00:0000:0100:01


In [2]:
TRAIN_DATA_DIR = "/kaggle/input/iasr-ds/VGG-Face2_sampled/train"
TEST_DATA_DIR = "/kaggle/input/iasr-ds/VGG-Face2_sampled/test"

In [29]:
!zip -r -q file.zip /kaggle/working

In [30]:
from IPython.display import FileLink
FileLink(r'file.zip')

/kaggle/working/file.zip

In [3]:
def load_and_preprocess_image(image_path):
    # Load the image using OpenCV
    img = cv2.imread(image_path, cv2.IMREAD_COLOR)
    
    # Convert BGR to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Resize the image to 256x256
    img = cv2.resize(img, (256, 256))
    
    # Normalize pixel values to [0, 1]
    img = img.astype(np.float32) / 255.0
    
    return img

In [4]:
train_image_paths = []
for person in tqdm(os.listdir(TRAIN_DATA_DIR)):
    for image_file in os.listdir(os.path.join(TRAIN_DATA_DIR, person)):
        train_image_paths.append(os.path.join(TRAIN_DATA_DIR, person, image_file))
len(train_image_paths)

100%|██████████| 8631/8631 [02:01<00:00, 70.85it/s]


314653

In [5]:
test_image_paths = []
for person in tqdm(os.listdir(TEST_DATA_DIR)):
    for image_file in os.listdir(os.path.join(TEST_DATA_DIR, person)):
        test_image_paths.append(os.path.join(TEST_DATA_DIR, person, image_file))
len(test_image_paths)

100%|██████████| 500/500 [00:10<00:00, 49.87it/s]


169396

In [6]:
import tensorflow as tf
tf.config.list_physical_devices(device_type=None)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
DETECTOR = MTCNN(device="GPU:0")

In [8]:
def detect_faces_mtcnn_batch(batch_images_np):
    batch_faces = []
    batch_status = []
    # Scale images to 0-255 and convert to uint8
    images_rgb = (batch_images_np * 255).astype('uint8')
    
    # Convert images to BGR format for MTCNN (if required by the detector)
    images_bgr = [cv2.cvtColor(img, cv2.COLOR_RGB2BGR) for img in images_rgb]
    
    # Detect faces in all images
    faces = [DETECTOR.detect_faces(img_bgr) for img_bgr in images_bgr]
    
    for img, face in zip(images_rgb, faces):
        if face and len(face) > 0:
            # Extract bounding box for the first detected face
            x, y, width, height = face[0]['box']
            x2, y2 = x + width, y + height
            
            # Crop and resize the face
            img_face = img[y:y2, x:x2]
            img_face = cv2.resize(img_face, (128, 128))
            status = True

            batch_faces.append(img_face)
        else:
            # If no face is detected, return a zeroed image
            # img_face = np.zeros((128, 128, 3), dtype=np.uint8)
            status = False
        
        # Append the processed face
        # batch_faces.append(img_face)
        batch_status.append(status)
    
    return batch_faces, batch_status

In [ ]:
!mkdir faces_ds
!mkdir faces_ds/train
!mkdir faces_ds/test

In [25]:
status_all = []

In [26]:
BATCH_SIZE = 512
len(train_image_paths), len(train_image_paths) / BATCH_SIZE

(314653, 614.556640625)

In [27]:
from tqdm import trange
for idx in trange(0, len(train_image_paths), BATCH_SIZE):
    paths = train_image_paths[idx:idx+BATCH_SIZE]
    imgs = [load_and_preprocess_image(path) for path in paths]
    imgs = np.array(imgs)
    batch_faces, batch_status = detect_faces_mtcnn_batch(imgs)
    status_all.extend(batch_status)
    
    for img, path in zip(batch_faces, paths):
        new_folder = "faces_ds/train/" + path.split("/")[-2]
        os.makedirs(new_folder, exist_ok=True)
        new_path = "faces_ds/train/" + path.split("/")[-2] + "/" + path.split("/")[-1]
        cv2.imwrite(new_path, img)

100%|██████████| 615/615 [10:19:40<00:00, 60.46s/it] 


In [28]:
sum(status_all) / len(status_all)

0.8592290554992325

In [11]:
status_all = []

In [12]:
from tqdm import trange
for idx in trange(0, len(test_image_paths), BATCH_SIZE):
    paths = test_image_paths[idx:idx+BATCH_SIZE]
    imgs = [load_and_preprocess_image(path) for path in paths]
    imgs = np.array(imgs)
    batch_faces, batch_status = detect_faces_mtcnn_batch(imgs)
    status_all.extend(batch_status)
    
    for img, path in zip(batch_faces, paths):
        new_folder = "faces_ds/test/" + path.split("/")[-2]
        os.makedirs(new_folder, exist_ok=True)
        new_path = "faces_ds/test/" + path.split("/")[-2] + "/" + path.split("/")[-1]
        cv2.imwrite(new_path, img)

100%|██████████| 331/331 [5:33:25<00:00, 60.44s/it]  


In [13]:
sum(status_all) / len(status_all)

0.865221138633734